# Introduction
Memory profile test in Jupyter

### Imports
Import libraries and write settings here.

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2


In [4]:
!free -l

              total        used        free      shared  buff/cache   available
Mem:       65596060    10920752    39906072     2160932    14769236    51863116
Low:       65596060    25689988    39906072
High:             0           0           0
Swap:     134217724           0   134217724


# mprun


In [88]:
#You can install memory_profiler with conda install memory_profiler.
!conda install memory_profiler -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.5

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [89]:
%load_ext memory_profiler
#you must load this extension

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [90]:
%%file mprun_demo1.py
import gc
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
        del L # remove reference to L
#         del total#cannot do this here, lose the var and cannot write to it 2nd time thru loop
#         gc.collect()
    return total

Overwriting mprun_demo1.py


In [94]:
%%file mprun_demo1.py
import gc
def sum_of_lists(N):
#     gc.collect()
    bigl=[]
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
        bigl.append(L)
#         del L # remove reference to L
#         gc.collect()
    del bigl
    del total
    gc.collect()
    return 

def callit(numb):
    sum_of_lists(numb)

Overwriting mprun_demo1.py


In [95]:
from mprun_demo1 import sum_of_lists
from mprun_demo1 import callit
%mprun -f callit -f sum_of_lists callit(100000)

In [96]:
#see what the source is for a function
import inspect
inspect.getsource(sum_of_lists)

'def sum_of_lists(N):\n#     gc.collect()\n    bigl=[]\n    total = 0\n    for i in range(5):\n        L = [j ^ (j >> i) for j in range(N)]\n        total += sum(L)\n        bigl.append(L)\n#         del L # remove reference to L\n#         gc.collect()\n    del bigl\n    del total\n    gc.collect()\n    return \n'

In [ ]:
#help
%mprun?

# memit

In [111]:
%memit?

In [115]:
%memit -c callit (100000)

peak memory: 235.98 MiB, increment: 107.01 MiB


# Ref Counts
https://stackoverflow.com/questions/7544395/sys-getrefcount-return-value/7544636
Large integer objects are not reused by the interpretor, so you get two distinct objects:
a = 11111
b = 11111
id(a)
40351656
id(b)
40351704
sys.getrefcount(11111) always returns the same number because it measures the reference count of a fresh object.

For small integers, Python always reuses the same object:
sys.getrefcount(1)
73
Usually you would get only one reference to a new object:
sys.getrefcount(object())
1

In [101]:
#check ref counts
tmp1=127888907
# del tmp1

In [106]:
#globals() #too much info
import sys
sys.getrefcount(tmp1)


2

In [108]:
a=tmp1

In [105]:
del a

In [107]:
#lets see if they are the same object
id(tmp1)

140476517299536

In [109]:
id(a)

140476517299536

In [116]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          64058        9221       32890        1778       21947       52359
Swap:        131071           0      131071
